## Quick start: Custom Handler Example

Here we will look into custom event handler implementation. Unlike the Basic Usage example, we will not pay much attention to anything behind the handler.

Handler is a special object inherited from dxfeed.EventHandler class. It is attached to the Subscription. When an event comes, internal structures call the update method of your handler with event as a parameter.  event is a list with data specific to each event type. For example, for the Trade event type: 'Symbol', 'Price', 'ExchangeCode', 'Size', 'Tick', 'Change', 'DayVolume', 'Time', 'IsETH'. 

After adding symbols or attaching a default listener (what is actually done implicitly in the first case) list of one-word descriptions of event data is stored in columns field of your handler object, attached to Subscription.

In this example, we will implement the event handler that prints price and volume changes for candle ask events. It also prints average volume change for the last 10 ask events.


### Import package

In [ ]:
import dxfeed as dx
from datetime import datetime  # for timed subscription
from dateutil.relativedelta import relativedelta
import pandas as pd

### Configure and create connection with Endpoint class
Create instance of Endpoint class which will connect provided address. 

In [ ]:
endpoint = dx.Endpoint('demo.dxfeed.com:7300')

### Configure and create subscription
You should specify event type. For timed subscription (conflated stream) you should also provide time to start subscription from.

In [ ]:
candle_sub = endpoint.create_subscription('Candle', date_time=datetime.now() - relativedelta(minutes=30))

In [ ]:
class DiffHandler(dx.EventHandler):
    def __init__(self):
        self.__prev_open = 0
        self.__prev_high = 0
        self.__prev_low = 0
        self.__prev_close = 0
        self.__prev_volume = 0
        self.volume_changes = list()
        self.counter = 0
        
    def update(self, event):
        if pd.notna(event.ask_volume):  # AskVolume not nan
            print(f'Symbol: {event.symbol}')
            print(f'Open changed by: {event.open - self.__prev_open}')
            self.__prev_open = event.open
            print(f'High changed by: {event.high - self.__prev_high}')
            self.__prev_high = event.high
            print(f'Low changed by: {event.low - self.__prev_low}')
            self.__prev_low = event.low
            print(f'Close changed by: {event.close - self.__prev_close}')
            self.__prev_close = event.close
            # Volume logic
            vol_change = event.ask_volume - self.__prev_volume
            self.volume_changes.append(vol_change)
            self.counter +=1
            print(f'Volume changed by: {vol_change}, from {self.__prev_volume}, to {event.ask_volume}')
            self.__prev_volume = event.ask_volume
            print(f'Ask events prcessed: {self.counter}')
            print('-------------------')
            if self.counter % 10 == 0:
                print(f'Average volume change for 10 past ask events is: {sum(self.volume_changes) / len(self.volume_changes)}')
                self.volume_changes.clear()
                print('-------------------')

In [ ]:
handler = DiffHandler()
candle_sub.set_event_handler(handler).add_symbols(['AAPL'])

### Close subscription

In [ ]:
candle_sub.close_subscription()

### Close connection

In [ ]:
endpoint.close_connection()
print(f'Connection status: {endpoint.connection_status}')